In [1]:
# import resources
%matplotlib inline

from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.optim as optim
import requests
from torchvision import transforms, models

In [3]:
## Load in VGG19 (features)

#VGG19 is split into two portions:
#* `vgg19.features`, which are all the convolutional and pooling layers
#* `vgg19.classifier`, which are the three linear, classifier layers at the end

#We only need the `features` portion, which we're going to load in and "freeze" the weights of, below.

In [4]:
# get the "features" portion of VGG19 (we will not need the "classifier" portion)
vgg = models.vgg19(pretrained=True).features

# freeze all VGG parameters since we're only optimizing the target image
for param in vgg.parameters():
    param.requires_grad_(False)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /home/shreyansh/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [02:35<00:00, 3.70MB/s]
